In [1]:

import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from tqdm import tqdm


In [2]:

def fetch_uranium_stocks_data(stock):
    periods = ["1d", "5d", "1mo", "3mo", "6mo", "1y", "5y"]
    data_dict = {}
    for period in periods:
        data = yf.download(stock, period=period, group_by='ticker', progress=False)
        data_dict[period] = data
    return data_dict


In [3]:

def plot_refined_single_stock_period(data_dicts, uranium_stocks):
    fig = go.Figure()

    periods = ["1d", "5d", "1mo", "3mo", "6mo", "1y", "5y"]

    for stock in uranium_stocks:
        data_dict = data_dicts[stock]
        for period, data in data_dict.items():
            fig.add_trace(go.Candlestick(x=data.index,
                                         open=data['Open'],
                                         high=data['High'],
                                         low=data['Low'],
                                         close=data['Close'],
                                         visible=False,
                                         name=f"{stock} - {period}"))

    fig.data[uranium_stocks.index(uranium_stocks[0]) * len(periods)].visible = True

    fig.update_layout(title=f'Candlestick chart - {uranium_stocks[0]} - 1mo',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      xaxis_rangeslider_visible=False)

    stock_buttons = []
    for i, stock in enumerate(uranium_stocks):
        visible = [False] * len(fig.data)
        for j in range(len(periods)):
            if (i * len(periods) + j) < len(fig.data):
                visible[i * len(periods) + j] = True
        stock_buttons.append(dict(label=stock,
                                  method="update",
                                  args=[{"visible": visible}]))

    period_buttons = []
    for i, period in enumerate(periods):
        visible = [False] * len(fig.data)
        for j in range(len(uranium_stocks)):
            if (j * len(periods) + i) < len(fig.data):
                visible[j * len(periods) + i] = True
        period_buttons.append(dict(label=period,
                                   method="update",
                                   args=[{"visible": visible}]))

    fig.update_layout(updatemenus=[
        dict(buttons=stock_buttons,
             direction="down",
             pad={"r": 10, "t": 10},
             showactive=True,
             x=0.1,
             xanchor="left",
             y=1.15,
             yanchor="top"),
        dict(buttons=period_buttons,
             direction="down",
             pad={"r": 10, "t": 10},
             showactive=True,
             x=0.3,
             xanchor="left",
             y=1.15,
             yanchor="top")
    ])

    return fig


In [5]:

if __name__ == "__main__":
    uranium_stocks = [
        "CCJ", "UUUU", "URG", "UEC", "DNN", "UROY", "LEU", "WWR", "NXE", "UWEFF", "AZZUF", "BKYI", "FCUUF", "GVXXF", "PALAF"
    ]

    data_dicts = {}
    for stock in tqdm(uranium_stocks, desc="Fetching stock data"):
        data_dicts[stock] = fetch_uranium_stocks_data(stock)

    fig = plot_refined_single_stock_period(data_dicts, uranium_stocks)
    fig.show()


Fetching stock data:  40%|████      | 6/15 [00:16<00:25,  2.81s/it]


KeyboardInterrupt: 